# Quantum Game of Life Hamiltonian in PennyLane

**Tutorial Notebook**

This notebook walks through the construction and simulation of the Quantum Game of Life (QGoL) Hamiltonian using PennyLane.

**Based on:** Faux, D. (2019). "The semi-quantum game of life." [arXiv:1902.07835](https://arxiv.org/abs/1902.07835)

---

## Table of Contents

1. [Introduction & Setup](#introduction)
2. [Understanding Operator Mappings](#mappings)
3. [Building a Simple Hamiltonian](#simple-hamiltonian)
4. [Time Evolution Methods](#time-evolution)
5. [Neighbor Counting Projectors](#neighbor-counting)
6. [Full QGoL Implementation](#full-implementation)
7. [Analysis and Visualization](#analysis)
8. [Advanced Examples](#advanced)

---

## 1. Introduction & Setup <a name="introduction"></a>

The Quantum Game of Life extends Conway's cellular automaton to quantum mechanics. Cells exist in superposition states and evolve under the Hamiltonian:

$$H = \sum_{i} (b_i + b_i^\dagger) \cdot (N_i^{(2)} + N_i^{(3)})$$

Where:
- $(b_i + b_i^\dagger)$ is the state flip operator
- $N_i^{(k)}$ are neighbor counting projectors

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install pennylane numpy matplotlib

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from itertools import product, combinations

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("PennyLane version:", qml.__version__)
print("Setup complete! ✓")

---

## 2. Understanding Operator Mappings <a name="mappings"></a>

In the QGoL, we map quantum operators to Pauli operators:

| Quantum Operator | Physical Meaning | Pauli Representation | PennyLane |
|-----------------|------------------|---------------------|----------|
| $b_i + b_i^\dagger$ | State flip (dead ↔ alive) | $\sigma_x^i$ | `qml.PauliX(i)` |
| $n_j = b_j^\dagger b_j$ | Number operator (alive?) | $\frac{1}{2}(I - Z_j)$ | `0.5*(I(j) - Z(j))` |
| $1 - n_j$ | Empty operator (dead?) | $\frac{1}{2}(I + Z_j)$ | `0.5*(I(j) + Z(j))` |

Let's verify these mappings:

In [ ]:
print("=" * 70)
print("Operator Mappings Verification")
print("=" * 70)

# Create a simple device
dev = qml.device('default.qubit', wires=2)

# Test 1: Pauli X flips states
@qml.qnode(dev)
def test_pauli_x():
    # Start in |0⟩ (dead)
    qml.PauliX(0)  # Apply X (flip)
    return qml.probs(wires=0)

probs = test_pauli_x()
print("\n1. Pauli X as state flip operator:")
print(f"   Starting in |0⟩, after X: |0⟩={probs[0]:.1f}, |1⟩={probs[1]:.1f}")
print(f"   ✓ X flips |0⟩ → |1⟩")

# Test 2: Number operator projects onto |1⟩
print("\n2. Number operator n = ½(I - Z):")
print("   For state |0⟩: n|0⟩ = 0|0⟩ (eigenvalue 0)")
print("   For state |1⟩: n|1⟩ = 1|1⟩ (eigenvalue 1)")
print("   ✓ n projects onto the 'alive' state")

# Test 3: Empty operator projects onto |0⟩
print("\n3. Empty operator (1-n) = ½(I + Z):")
print("   For state |0⟩: (1-n)|0⟩ = 1|0⟩ (eigenvalue 1)")
print("   For state |1⟩: (1-n)|1⟩ = 0|1⟩ (eigenvalue 0)")
print("   ✓ (1-n) projects onto the 'dead' state")

print("\n" + "=" * 70)

---

## 3. Building a Simple Hamiltonian <a name="simple-hamiltonian"></a>

Let's start with a minimal 2-qubit example. We'll create a Hamiltonian where qubit 0 flips when qubit 1 is alive:

$$H = X_0 \otimes n_1 = X_0 \otimes \frac{1}{2}(I_1 - Z_1)$$

Expanding:
$$H = \frac{1}{2}X_0 \otimes I_1 - \frac{1}{2}X_0 \otimes Z_1$$

In [ ]:
print("=" * 70)
print("Simple 2-Qubit Hamiltonian")
print("=" * 70)

print("\nBuilding H = X₀ ⊗ n₁ where n₁ = ½(I₁ - Z₁)")
print("This means: qubit 0 flips when qubit 1 is alive\n")

# Define the Hamiltonian
coeffs = [0.5, -0.5]
obs = [
    qml.PauliX(0) @ qml.Identity(1),  # ½X₀ ⊗ I₁
    qml.PauliX(0) @ qml.PauliZ(1)      # -½X₀ ⊗ Z₁
]

H_simple = qml.Hamiltonian(coeffs, obs)

print("Hamiltonian terms:")
for c, op in zip(H_simple.coeffs, H_simple.ops):
    print(f"  {c:+.2f} × {op}")

print("\n" + "=" * 70)

---

## 4. Time Evolution Methods <a name="time-evolution"></a>

We can evolve quantum states under a Hamiltonian using:

1. **Trotterization** (approximate): $U(t) \approx [e^{-iH_1\Delta t} \cdots e^{-iH_n\Delta t}]^n$
2. **Exact Evolution** (for small systems): $U(t) = e^{-iHt}$

Let's compare both methods:

In [ ]:
print("=" * 70)
print("Time Evolution Comparison")
print("=" * 70)

dev = qml.device("default.qubit", wires=2)

# Method 1: Trotterization
@qml.qnode(dev)
def evolve_trotter(t, n_steps):
    """Evolve using TrotterProduct (approximate)."""
    # Start in state |10⟩ (qubit 0 alive, qubit 1 dead)
    qml.PauliX(wires=0)
    
    # Apply Trotterized time evolution
    qml.TrotterProduct(H_simple, t, n=n_steps)
    
    return qml.probs(wires=[0, 1])

# Method 2: Exact evolution
@qml.qnode(dev)
def evolve_exact(t):
    """Evolve using exact evolution."""
    # Start in state |10⟩
    qml.PauliX(wires=0)
    
    # Apply exact time evolution
    qml.ApproxTimeEvolution(H_simple, t, n=1)
    
    return qml.probs(wires=[0, 1])

# Evolve for time t = 1.0
time = 1.0
trotter_steps = 20

print(f"\nInitial state: |10⟩ (qubit 0 alive, qubit 1 dead)")
print(f"Evolution time: t = {time}")
print(f"Trotter steps: {trotter_steps}\n")

probs_trotter = evolve_trotter(time, trotter_steps)
probs_exact = evolve_exact(time)

states = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']

print("Results:")
print(f"{'State':<8} {'Trotterization':<18} {'Exact':<18} {'Difference':<12}")
print("-" * 60)
for state, p_trot, p_exact in zip(states, probs_trotter, probs_exact):
    diff = abs(p_trot - p_exact)
    print(f"{state:<8} {p_trot:<18.6f} {p_exact:<18.6f} {diff:<12.6f}")

max_diff = np.max(np.abs(probs_trotter - probs_exact))
print(f"\nMaximum difference: {max_diff:.8f}")
print("Note: Increasing Trotter steps reduces the difference")

print("\n" + "=" * 70)

### Visualizing Time Evolution

Let's see how the state evolves over time:

In [ ]:
# Simulate evolution at multiple time points
time_points = np.linspace(0, 5, 50)
evolution_data = []

for t in time_points:
    probs = evolve_exact(t)
    evolution_data.append(probs)

evolution_data = np.array(evolution_data)

# Plot
plt.figure(figsize=(12, 5))

for i, state in enumerate(states):
    plt.plot(time_points, evolution_data[:, i], label=state, linewidth=2)

plt.xlabel('Time', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.title('State Evolution Under Simple QGoL Hamiltonian', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("The plot shows how quantum superposition evolves over time.")
print("Notice the oscillatory behavior - a signature of quantum dynamics!")

---

## 5. Neighbor Counting Projectors <a name="neighbor-counting"></a>

The key to QGoL is the neighbor counting projector $N_i^{(k)}$ which equals 1 when site $i$ has exactly $k$ alive neighbors.

For a site with neighbors $\{1, 2, \ldots, n\}$:

$$N_i^{(k)} = \sum_{\substack{S \subseteq \{1,\ldots,n\} \\ |S| = k}} \prod_{j \in S} n_j \prod_{j \notin S} (1-n_j)$$

Let's build $N_0^{(2)}$ for a 3-qubit system where qubit 0 has neighbors 1 and 2:

In [ ]:
print("=" * 70)
print("Neighbor Counting Projector: N₀⁽²⁾")
print("=" * 70)

print("\nFor a 3-qubit system where qubit 0 has neighbors {1, 2}")
print("Building N₀⁽²⁾ = n₁ · n₂ (both neighbors alive)\n")

print("Step 1: Substitute n_j = ½(I - Z_j)")
print("  N₀⁽²⁾ = [½(I₁ - Z₁)] · [½(I₂ - Z₂)]")

print("\nStep 2: Expand the product")
print("  N₀⁽²⁾ = ¼[(I₁ - Z₁)(I₂ - Z₂)]")
print("       = ¼[I₁I₂ - I₁Z₂ - Z₁I₂ + Z₁Z₂]")

print("\nStep 3: Build the Pauli terms")

# Build the neighbor counting projector
coeffs_n2 = [0.25, -0.25, -0.25, 0.25]
obs_n2 = [
    qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2),
    qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2),
    qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2),
    qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
]

for c, op in zip(coeffs_n2, obs_n2):
    print(f"  {c:+.2f} × {op}")

print("\nStep 4: Multiply by X₀ to get QGoL term")
print("  H₀⁽²⁾ = X₀ ⊗ N₀⁽²⁾")

obs_h = [qml.PauliX(0) @ obs for obs in obs_n2]
H_neighbor = qml.Hamiltonian(coeffs_n2, obs_h)

print("\nFinal Hamiltonian term (first 3 terms):")
for i, (c, op) in enumerate(zip(H_neighbor.coeffs[:3], H_neighbor.ops[:3])):
    print(f"  {c:+.2f} × {op}")
print("  ...")

print("\n✓ This term flips qubit 0 when it has exactly 2 alive neighbors")
print("\n" + "=" * 70)

### Testing the Neighbor Projector

Let's verify that $N_0^{(2)}$ actually counts neighbors correctly:

In [ ]:
print("Testing N₀⁽²⁾ on different states:\n")

dev3 = qml.device('default.qubit', wires=3)

# Test function
def test_neighbor_projector(state_prep_func, state_name):
    @qml.qnode(dev3)
    def circuit():
        state_prep_func()
        return qml.expval(qml.Hamiltonian(coeffs_n2, obs_n2))
    
    result = circuit()
    return result

# Test cases
test_cases = [
    (lambda: None, "|000⟩", "0 neighbors alive", 0.0),
    (lambda: qml.PauliX(1)(), "|010⟩", "1 neighbor alive", 0.0),
    (lambda: [qml.PauliX(1)(), qml.PauliX(2)()], "|011⟩", "2 neighbors alive", 1.0),
    (lambda: [qml.PauliX(0)(), qml.PauliX(1)()], "|110⟩", "1 neighbor alive", 0.0),
]

print(f"{'State':<10} {'Description':<25} {'N₀⁽²⁾ value':<15} {'Expected':<10}")
print("-" * 65)

for prep, state, desc, expected in test_cases:
    result = test_neighbor_projector(prep, state)
    match = "✓" if abs(result - expected) < 0.01 else "✗"
    print(f"{state:<10} {desc:<25} {result:<15.4f} {expected:<10.1f}  {match}")

print("\n✓ The projector correctly identifies when qubit 0 has 2 alive neighbors!")

---

## 6. Full QGoL Implementation <a name="full-implementation"></a>

Now let's implement the complete QGoL Hamiltonian for a small grid. We'll create a class that handles everything:

In [ ]:
class QuantumGameOfLife:
    """
    Complete implementation of the Quantum Game of Life Hamiltonian.
    """
    
    def __init__(self, grid_size=(2, 2), periodic=True):
        self.rows, self.cols = grid_size
        self.n_qubits = self.rows * self.cols
        self.periodic = periodic
        self.dev = qml.device('default.qubit', wires=self.n_qubits)
        
    def coord_to_qubit(self, row, col):
        """Convert 2D grid coordinates to qubit index."""
        return row * self.cols + col
    
    def qubit_to_coord(self, qubit):
        """Convert qubit index to 2D grid coordinates."""
        return qubit // self.cols, qubit % self.cols
        
    def get_neighbors(self, row, col):
        """Get the qubit indices of neighboring cells (Moore neighborhood)."""
        neighbors = []
        
        for dr, dc in [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]:
            if self.periodic:
                new_row = (row + dr) % self.rows
                new_col = (col + dc) % self.cols
            else:
                new_row = row + dr
                new_col = col + dc
                if new_row < 0 or new_row >= self.rows or new_col < 0 or new_col >= self.cols:
                    continue
                    
            neighbors.append(self.coord_to_qubit(new_row, new_col))
            
        return neighbors
    
    def build_neighbor_projector(self, site, k):
        """Build the neighbor counting projector N_i^(k)."""
        row, col = self.qubit_to_coord(site)
        neighbors = self.get_neighbors(row, col)
        n_neighbors = len(neighbors)
        
        if k > n_neighbors:
            return [], []
        
        coeffs = []
        obs = []
        
        # For each combination of k neighbors that are alive
        for alive_neighbors in combinations(neighbors, k):
            alive_set = set(alive_neighbors)
            coeff_product = (0.5) ** n_neighbors
            
            # Generate all possible combinations of I and Z for each neighbor
            for z_pattern in product([0, 1], repeat=n_neighbors):
                pauli_ops = []
                term_coeff = coeff_product
                
                for idx, neighbor in enumerate(neighbors):
                    if neighbor in alive_set:
                        if z_pattern[idx] == 1:
                            pauli_ops.append(qml.PauliZ(neighbor))
                            term_coeff *= -1
                    else:
                        if z_pattern[idx] == 1:
                            pauli_ops.append(qml.PauliZ(neighbor))
                
                if len(pauli_ops) == 0:
                    obs_term = qml.Identity(site)
                else:
                    obs_term = pauli_ops[0]
                    for op in pauli_ops[1:]:
                        obs_term = obs_term @ op
                
                coeffs.append(term_coeff)
                obs.append(obs_term)
        
        return coeffs, obs
    
    def build_hamiltonian(self):
        """Build the full QGoL Hamiltonian: H = Σᵢ Xᵢ · (Nᵢ⁽²⁾ + Nᵢ⁽³⁾)."""
        all_coeffs = []
        all_obs = []
        
        for site in range(self.n_qubits):
            for k in [2, 3]:
                proj_coeffs, proj_obs = self.build_neighbor_projector(site, k)
                
                for c, obs in zip(proj_coeffs, proj_obs):
                    full_obs = qml.PauliX(site) @ obs
                    all_coeffs.append(c)
                    all_obs.append(full_obs)
        
        return qml.Hamiltonian(all_coeffs, all_obs)
    
    def evolve(self, H, time, n_steps, initial_state=None):
        """Evolve the QGoL state using Trotterization."""
        @qml.qnode(self.dev)
        def circuit():
            if initial_state is not None:
                qml.QubitStateVector(initial_state, wires=range(self.n_qubits))
            qml.TrotterProduct(H, time, n=n_steps)
            return qml.probs(wires=range(self.n_qubits))
        
        return circuit()

print("✓ QuantumGameOfLife class defined!")

### Creating a 2×2 QGoL Grid

In [ ]:
print("=" * 70)
print("Building 2×2 QGoL Grid")
print("=" * 70)

# Create a 2×2 grid
qgol = QuantumGameOfLife(grid_size=(2, 2), periodic=True)

print(f"\nGrid configuration:")
print(f"  Size: {qgol.rows}×{qgol.cols}")
print(f"  Number of qubits: {qgol.n_qubits}")
print(f"  Boundary conditions: {'Periodic' if qgol.periodic else 'Fixed'}")

# Show the grid layout
print(f"\n  Grid layout:")
print(f"  [0, 1]")
print(f"  [2, 3]")

# Show neighbor relationships
print(f"\n  Neighbor relationships (with periodic boundaries):")
for qubit in range(qgol.n_qubits):
    row, col = qgol.qubit_to_coord(qubit)
    neighbors = qgol.get_neighbors(row, col)
    print(f"    Qubit {qubit}: neighbors = {neighbors}")

print("\n" + "=" * 70)

### Building the Full Hamiltonian

In [ ]:
print("Building the complete QGoL Hamiltonian...\n")

H_full = qgol.build_hamiltonian()

print(f"✓ Hamiltonian constructed!")
print(f"  Total number of terms: {len(H_full.ops)}")
print(f"  Structure: H = Σᵢ Xᵢ ⊗ (Nᵢ⁽²⁾ + Nᵢ⁽³⁾)")

print(f"\nFirst 5 Hamiltonian terms:")
for i in range(min(5, len(H_full.ops))):
    print(f"  {H_full.coeffs[i]:+.4f} × {H_full.ops[i]}")
print("  ...")

print(f"\nNote: For a 2×2 grid with periodic boundaries:")
print(f"  - Each site has 3 neighbors")
print(f"  - Each site contributes C(3,2)×2³ + C(3,3)×2³ = 32 terms")
print(f"  - Total: 4 sites × 32 terms/site = 128 terms")

---

## 7. Analysis and Visualization <a name="analysis"></a>

Let's simulate the QGoL evolution and analyze the results:

In [ ]:
print("=" * 70)
print("QGoL Simulation")
print("=" * 70)

# Prepare initial state - superposition
print("\n1. Preparing initial state...")
initial_state = np.zeros(2**qgol.n_qubits)
initial_state[0] = 0.7   # |0000⟩ (all dead)
initial_state[5] = 0.5   # |0101⟩ (checkerboard)
initial_state[10] = 0.3  # |1010⟩ (checkerboard)
initial_state[15] = 0.4  # |1111⟩ (all alive)
initial_state = initial_state / np.linalg.norm(initial_state)

print(f"   Initial state: superposition of {np.count_nonzero(initial_state)} basis states")

# Evolve the system
print("\n2. Evolving the system...")
time = 2.0
n_steps = 20
probs_final = qgol.evolve(H_full, time, n_steps, initial_state)

print(f"   Evolution time: t = {time}")
print(f"   Trotter steps: {n_steps}")

# Display results
print("\n3. Final state probabilities (top 10):")
print(f"\n   {'State':<12} {'Binary':<10} {'Probability':<15} {'Bar'}")
print("   " + "-" * 60)

# Get top 10 states
top_indices = np.argsort(probs_final)[-10:][::-1]

for idx in top_indices:
    binary = format(idx, f'0{qgol.n_qubits}b')
    prob = probs_final[idx]
    bar = '█' * int(prob * 50)
    print(f"   |{binary}⟩  {binary:>8}  {prob:>12.6f}  {bar}")

print("\n" + "=" * 70)

### Analyzing Liveness Distribution

In the Faux (2019) paper, the "quantum cloud" is characterized by a specific liveness distribution. Let's analyze ours:

In [ ]:
def analyze_liveness(probs, n_qubits):
    """Calculate liveness statistics from probability distribution."""
    liveness_dist = np.zeros(n_qubits + 1)
    
    for state_idx, prob in enumerate(probs):
        n_alive = bin(state_idx).count('1')
        liveness_dist[n_alive] += prob
    
    mean_liveness = sum(i * liveness_dist[i] for i in range(len(liveness_dist)))
    variance = sum((i - mean_liveness)**2 * liveness_dist[i] for i in range(len(liveness_dist)))
    std_liveness = np.sqrt(variance)
    
    return liveness_dist, mean_liveness, std_liveness

# Analyze initial and final states
initial_probs = initial_state ** 2
liveness_init, mean_init, std_init = analyze_liveness(initial_probs, qgol.n_qubits)
liveness_final, mean_final, std_final = analyze_liveness(probs_final, qgol.n_qubits)

print("Liveness Analysis:\n")
print(f"Initial state:")
print(f"  Mean liveness: {mean_init:.4f}")
print(f"  Std liveness:  {std_init:.4f}")
print(f"\nFinal state (t={time}):")
print(f"  Mean liveness: {mean_final:.4f}")
print(f"  Std liveness:  {std_final:.4f}")

print(f"\nPaper values (for larger grids, longer times):")
print(f"  Mean liveness: 0.3480 ± 0.0001")
print(f"  Std liveness:  0.0071")

# Plot liveness distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
x = np.arange(qgol.n_qubits + 1)
width = 0.35
plt.bar(x - width/2, liveness_init, width, label='Initial', alpha=0.7)
plt.bar(x + width/2, liveness_final, width, label=f'Final (t={time})', alpha=0.7)
plt.xlabel('Number of Alive Cells', fontsize=11)
plt.ylabel('Probability', fontsize=11)
plt.title('Liveness Distribution', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, axis='y')

plt.subplot(1, 2, 2)
# Visualize grid probabilities
marginal_probs = np.zeros(qgol.n_qubits)
for state_idx, prob in enumerate(probs_final):
    binary = format(state_idx, f'0{qgol.n_qubits}b')
    for qubit in range(qgol.n_qubits):
        if binary[qubit] == '1':
            marginal_probs[qubit] += prob

grid_probs = marginal_probs.reshape(qgol.rows, qgol.cols)
im = plt.imshow(grid_probs, cmap='RdYlGn', vmin=0, vmax=1, interpolation='nearest')
plt.colorbar(im, label='P(alive)')
plt.title(f'Cell Liveness (t={time})', fontsize=13)
plt.xlabel('Column', fontsize=11)
plt.ylabel('Row', fontsize=11)

for i in range(qgol.rows):
    for j in range(qgol.cols):
        plt.text(j, i, f'{grid_probs[i, j]:.2f}', 
                ha='center', va='center', color='black', fontsize=12)

plt.tight_layout()
plt.show()

### Time Evolution Trajectory

In [ ]:
print("Simulating time evolution trajectory...\n")

time_points = np.linspace(0, 5, 21)
mean_liveness_trajectory = []
std_liveness_trajectory = []

for t in time_points:
    probs = qgol.evolve(H_full, t, n_steps=20, initial_state=initial_state)
    _, mean, std = analyze_liveness(probs, qgol.n_qubits)
    mean_liveness_trajectory.append(mean)
    std_liveness_trajectory.append(std)

# Plot
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(time_points, mean_liveness_trajectory, 'b-o', linewidth=2, markersize=6)
plt.axhline(y=0.348, color='r', linestyle='--', linewidth=2, label='Paper value')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Mean Liveness', fontsize=12)
plt.title('Evolution of Mean Liveness', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(time_points, std_liveness_trajectory, 'g-o', linewidth=2, markersize=6)
plt.axhline(y=0.0071, color='r', linestyle='--', linewidth=2, label='Paper value')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Std Liveness', fontsize=12)
plt.title('Evolution of Liveness Std Dev', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Exact convergence to paper values requires:")
print("  - Larger grids (the paper used much larger systems)")
print("  - Longer evolution times")
print("  - Appropriate initial conditions")
print("\nOur 2×2 system demonstrates the principles but is too small")
print("to fully reproduce the 'quantum cloud' phenomenon.")

---

## 8. Advanced Examples <a name="advanced"></a>

### Example 1: Different Initial States

In [ ]:
print("=" * 70)
print("Comparing Different Initial States")
print("=" * 70)

# Define different initial states
initial_states = {
    'All Dead': np.array([1.0 if i == 0 else 0.0 for i in range(2**qgol.n_qubits)]),
    'All Alive': np.array([1.0 if i == 15 else 0.0 for i in range(2**qgol.n_qubits)]),
    'Superposition': np.ones(2**qgol.n_qubits) / np.sqrt(2**qgol.n_qubits),
}

time = 2.0
results = {}

for name, state in initial_states.items():
    probs = qgol.evolve(H_full, time, 20, state)
    _, mean, std = analyze_liveness(probs, qgol.n_qubits)
    results[name] = (mean, std, probs)
    print(f"\n{name}:")
    print(f"  Final mean liveness: {mean:.4f}")
    print(f"  Final std liveness:  {std:.4f}")

# Visualize
plt.figure(figsize=(15, 4))

for idx, (name, (mean, std, probs)) in enumerate(results.items(), 1):
    plt.subplot(1, 3, idx)
    
    # Get marginal probabilities
    marginal = np.zeros(qgol.n_qubits)
    for state_idx, prob in enumerate(probs):
        binary = format(state_idx, f'0{qgol.n_qubits}b')
        for qubit in range(qgol.n_qubits):
            if binary[qubit] == '1':
                marginal[qubit] += prob
    
    grid = marginal.reshape(qgol.rows, qgol.cols)
    im = plt.imshow(grid, cmap='RdYlGn', vmin=0, vmax=1, interpolation='nearest')
    plt.colorbar(im, label='P(alive)')
    plt.title(f'{name}\n(mean={mean:.3f})', fontsize=11)
    
    for i in range(qgol.rows):
        for j in range(qgol.cols):
            plt.text(j, i, f'{grid[i, j]:.2f}', 
                    ha='center', va='center', color='black')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)

### Example 2: Varying Grid Sizes

Let's see how the Hamiltonian complexity scales with grid size:

In [ ]:
print("=" * 70)
print("Hamiltonian Scaling with Grid Size")
print("=" * 70)

grid_sizes = [(2, 2), (2, 3), (3, 3)]
scaling_data = []

print(f"\n{'Grid Size':<15} {'Qubits':<10} {'Hamiltonian Terms':<20}")
print("-" * 50)

for grid_size in grid_sizes:
    qgol_temp = QuantumGameOfLife(grid_size=grid_size, periodic=True)
    H_temp = qgol_temp.build_hamiltonian()
    
    n_qubits = qgol_temp.n_qubits
    n_terms = len(H_temp.ops)
    
    print(f"{grid_size[0]}×{grid_size[1]:<12} {n_qubits:<10} {n_terms:<20}")
    scaling_data.append((n_qubits, n_terms))

# Plot scaling
plt.figure(figsize=(10, 5))

qubits = [d[0] for d in scaling_data]
terms = [d[1] for d in scaling_data]

plt.subplot(1, 2, 1)
plt.semilogy(qubits, terms, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Number of Qubits', fontsize=12)
plt.ylabel('Number of Hamiltonian Terms', fontsize=12)
plt.title('Hamiltonian Complexity Scaling', fontsize=13)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
state_space = [2**q for q in qubits]
plt.semilogy(qubits, state_space, 'ro-', linewidth=2, markersize=10)
plt.xlabel('Number of Qubits', fontsize=12)
plt.ylabel('Hilbert Space Dimension', fontsize=12)
plt.title('State Space Scaling', fontsize=13)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️  Note: Computational cost scales exponentially!")
print("   Stay below ~10 qubits for practical simulation.")
print("\n" + "=" * 70)

---

## Summary and Key Takeaways

### What We've Learned:

1. **Operator Mappings**: How to translate quantum operators to Pauli operators
   - $(b + b^\dagger) \rightarrow X$
   - $n = b^\dagger b \rightarrow \frac{1}{2}(I - Z)$

2. **Neighbor Counting**: How to build projectors that count alive neighbors
   - $N_i^{(k)}$ equals 1 when exactly $k$ neighbors are alive
   - Constructed from products of number operators

3. **Hamiltonian Construction**: Building the full QGoL Hamiltonian
   - $H = \sum_i X_i \otimes (N_i^{(2)} + N_i^{(3)})$
   - Results in many Pauli terms

4. **Time Evolution**: Two methods for simulating quantum dynamics
   - Trotterization: approximate but scalable
   - Exact: accurate but limited to small systems

5. **Analysis**: Understanding the "quantum cloud"
   - Characterized by liveness distribution
   - Emerges from chaotic quantum evolution

### Next Steps:

- Try different grid sizes (stay below 10 qubits!)
- Experiment with initial states
- Modify the rules (e.g., different neighbor counts)
- Compare with classical Game of Life
- Explore other cellular automata

### References:

- Faux, D. (2019). "The semi-quantum game of life." [arXiv:1902.07835](https://arxiv.org/abs/1902.07835)
- PennyLane Documentation: [https://pennylane.ai/](https://pennylane.ai/)

---

**Happy quantum computing! 🚀**